# Prerequisites

In [1]:
import numpy as np
import tables as tb
import os
import glob
import csv
import shutil
import tqdm as tq
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16,9]

from utils import *

class Config:
    pass

config = Config()
config.nbFrequencyBands = 29

config.rootPath = '/home/lagrange/data/storage/cense/confy/'
config.inputPath = config.rootPath+'raw/'
config.dataPath = config.rootPath+'data/'

config.sensorInfo = sensor_list('sensorList.csv')

config.month = ['january', 'march']
config.monthPath = ['01_02_2020', '04_05_2020']



Found 16 sensors.


# Generate dataset


In [2]:
class Sensor(tb.IsDescription):
  number = tb.Int32Col()
  id = tb.StringCol(100)
  lat = tb.Float64Col()
  lon = tb.Float64Col() 

In [ ]:
def generateDataset(config, mType):
    tmpFileName = config.dataPath+'tmpDa'
    datasetFileName = 'censeConfy'+config.month[mType]+'.h5'
    data = np.zeros((1000, config.nbFrequencyBands))
    dataTime = np.zeros((1000, 1))

    dryRun = False
    oneDay = True
    reDo = True
    
    if oneDay:
        dayLimit = 2
    else:
        dayLimit = 33

    if 'f' in locals():
      f.close()

    if reDo :
      fileName = config.dataPath+datasetFileName
      if dryRun:
        fileName = fileName.replace('.h5', 'DryRun.h5')
      if oneDay:
        fileName = fileName.replace('.h5', 'OneDay.h5')
      print(fileName)
      f = tb.open_file(fileName, mode='w')
      f.create_table('/', 'sensor', Sensor, 'Sensor information')
      time = f.create_group('/', 'time', 'time expressed in epoch')
      spectrum = f.create_group('/', 'spectrum', 'spectral data third octave bands fast (125ms)')
      for sCount, s in enumerate(config.sensorInfo): #tq.tqdm(enumerate(sensorInfo), total=len(sensorInfo)):
        print('Sensor '+str(sCount)+' / '+str(len(config.sensorInfo)))
        sensor = f.root.sensor.row
        sensor['id'] = s["sID"]
        sensor['lat'] = s["latGPS"]
        sensor['lon'] = s["lonGPS"]
        sensor['number'] = sCount
        sensor.append()
        arrayTime = f.create_earray(time, 'sensor'+str(sCount), tb.Float64Atom(), (0,))
        arraySpec = f.create_earray(spectrum, 'sensor'+str(sCount), tb.Float64Atom(), (0, config.nbFrequencyBands))
        #print('----- '+s["sID"]+' -----')
        if dryRun:
          if np.random.randint(1):
            data = np.zeros((10, 32))
            arrayTime.append(data[:, 0])
            arraySpec.append(data[:, 3:])
        else:
          fileNames = []
          for year in [2019, 2020]:
            for month in range(13):
              for day in range(dayLimit):
                for hour in range(25):
                  fileName = config.inputPath+config.monthPath[mType]+'/'+s["sID"]+'/'+str(year)+'/'+str(month)+'/'+str(day)+'/'+str(hour)+'.zip'
                  if os.path.exists(fileName):
                    fileNames.append(fileName)

          for fileName in tq.tqdm(fileNames, total=len(fileNames)):         
            shutil.copy(fileName, tmpFileName+'.zip')
            os.system('unzip -d /tmp '+tmpFileName+'.zip')
            csvFileName = os.path.basename(fileName).replace('zip', 'csv')
            with open('/tmp/'+csvFileName, 'r') as csvfileID:
              reader = csv.reader(csvfileID, delimiter=',')
              nbVec = (sum(1 for row in reader))
            with open('/tmp/'+csvFileName, 'r') as csvfileID: 
              reader = csv.reader(csvfileID, delimiter=',')
              data = np.zeros((nbVec, 32))
              rCount = 0
              for r, row in enumerate(reader):
                if r%4800<360: # every 10 minutes, for 45 seconds
                    data[rCount, :] = [float(s) for s in row]
                    rCount +=1
                    arrayTime.append(data[:, 0])
                    arraySpec.append(data[:, 3:])

              os.remove('/tmp/'+csvFileName)
            os.remove(tmpFileName+'.zip')
      f.root.sensor.flush()
      f.close()

tb.file._open_files.close_all()
generateDataset(config, 1)
generateDataset(config, 0)

Closing remaining open files:/home/lagrange/data/storage/cense/confy/data/censeConfymarchOneDay.h5...done


/home/lagrange/data/storage/cense/confy/data/censeConfymarchOneDay.h5


  0%|          | 0/24 [00:00<?, ?it/s]

Sensor 0 / 16


  4%|▍         | 1/24 [01:01<23:45, 61.97s/it]

# check dataset integrity

  

In [ ]:
print(len(sensorInfo))
f = tb.open_file(dataPath+datasetFileName, mode='r')
print(len(f.root.sensor))
print(sum(1 for x in f.root.time))
print(sum(1 for x in f.root.spectrum))
f.close()

# Check disk usage

In [ ]:
! du -sh {dataPath}

# remove sensors with low content

In [ ]:
f = tb.open_file(dataPath+datasetFileName, mode='a')
for d in f.root.spectrum: # loop over spectral data
  if d.shape[0]<2000000:
    for xCount, x in enumerate(f.root.sensor):   
      if x['number'] == int(d._v_name.replace('sensor', '')):
        f.root.sensor.remove_row(xCount) # remove column in table
    f.root.time._f_get_child(d._v_name)._f_remove() # remove time data
    d._f_remove() # remove spectral data
f.root.sensor.flush()
f.close()

# put all sensors at length

In [ ]:
f = tb.open_file(dataPath+datasetFileName, mode='a')
minSize = min([x.shape[0] for x in f.root.spectrum])
print(minSize)
minSize = 300000 # uncomment to generate small dataset
for s in f.root.sensor: # loop over sensors
  f.root.spectrum._f_get_child('sensor'+str(s['number'])).rename('tmp')
  f.create_array(f.root.spectrum, 'sensor'+str(s['number']), f.root.spectrum.tmp[:minSize])
  f.root.spectrum._f_get_child('tmp')._f_remove()
  print(f.root.spectrum._f_get_child('sensor'+str(s['number'])).shape)

  f.root.time._f_get_child('sensor'+str(s['number'])).rename('tmp')
  f.create_array(f.root.time, 'sensor'+str(s['number']), f.root.time.tmp[:minSize])
  f.root.time._f_get_child('tmp')._f_remove()
  print(f.root.time._f_get_child('sensor'+str(s['number'])).shape) 
f.close()

# Sanity check

min max, and average mean, std per block
per sensor and frequencyBand

In [ ]:
def sanityCheck(datasetPath, dryRun=False):

  f = tb.open_file(datasetPath, mode='r')
  stats = np.zeros((len(f.root.sensor), f.root.spectrum.sensor0.shape[1], 5))

  for dCount, data in tqdm(enumerate(f.root.spectrum), total=len(f.root.sensor)):
    if data.shape[0]>0:
      if dryRun:
        data = data[0:100, :]
      stats[dCount, :, 0] = np.amin(data, axis=0)
      stats[dCount, :, 1] = np.amax(data, axis=0)
      stats[dCount, :, 2] = np.mean(data, axis=0)
      stats[dCount, :, 3] = np.std(data, axis=0)
      stats[dCount, :, 4] = data.shape[0]
  f.close()

  return stats

stats = sanityCheck(dataPath+datasetFileName)

# Display statistics

In [ ]:
plt.rcParams["figure.figsize"] = [16,9]

f, axes = plt.subplots(2, 2)
statNames = ['Min', 'Max', 'Mean', 'Std']
axes = axes.flatten()
for aCount, ax in enumerate(axes):
  ax.set_title(statNames[aCount])
  i = ax.imshow(stats[:, :, aCount])
  f.colorbar(i, ax=ax)
  ax.set_xlabel('Frequency bands')
  ax.set_ylabel('Sensor id')

f.show()
f.savefig(localPath+'/figures/stats.png')

# check for discontinuity

In [ ]:
tolerance = 250

f = tb.open_file(dataPath+datasetFileName, mode='r')

nbDisc = np.zeros((len(f.root.sensor), 1))
nbVec = np.zeros((len(f.root.sensor), 1))

for tCount, t in tqdm(enumerate(f.root.time), total=len(f.root.sensor)):
  t0 = t[0]
  nbVec[tCount] = len(t)
  for t1 in t[1:]:
    if t1-t0 > tolerance:
      nbDisc[tCount] += 1
    t0=t1
f.close()

# display discontinuity per sensor

In [ ]:
fig, axes = plt.subplots(2, 1)

axes[0].plot(nbDisc/max(nbVec))
axes[0].set_xlabel('Sensor id')
axes[0].set_ylabel('# dicontinuities / # vectors')
axes[1].plot(nbVec)
axes[1].set_xlabel('Sensor id')
axes[1].set_ylabel('# vectors')

plt.show()
fig.savefig(localPath+'/figures/discontinuities.png')

# check for continuity

In [ ]:
f = tb.open_file(dataPath+datasetFileName, mode='r')
print(len(f.root.time.sensor13))
plt.plot(f.root.time.sensor13)

# curate sensor naming

In [ ]:
f = tb.open_file(dataPath+datasetFileName, mode='a')
for dCount, d in enumerate(f.root.sensor): # loop over sensor table
  print(d['number'])
  f.root.time._f_get_child('sensor'+str(d['number'])).rename('sensorTmp'+str(dCount))
  f.root.spectrum._f_get_child('sensor'+str(d['number'])).rename('sensorTmp'+str(dCount))
  d['number'] = dCount 
  d.update()

for d in f.root.spectrum:
  d.rename(d._v_name.replace('Tmp', ''))
for d in f.root.time:
  d.rename(d._v_name.replace('Tmp', ''))
f.close()

# Debug

# close all tables files
  

In [ ]:
tb.file._open_files.close_all()

# revert backup

In [ ]:
! rsync  --info=progress2 {dataPath+datasetFileName.replace('.h5', 'Bck.h5')} {dataPath+datasetFileName}

In [ ]:
f = tb.open_file(inputPath+datasetFileName, mode='r')
print(f.root.sensor[:]['number'])
print(f)
f.close()

In [ ]:
! ptrepack {dataPath+datasetFileName} {dataPath+'tmp.h5'}
! mv {dataPath+'tmp.h5'} {dataPath+datasetFileName}

In [ ]:
stats


In [ ]:
np.save('/experiments/data/global/censeDomainAdaptation/models', stats)

In [ ]:
28800*125/1000/60

In [ ]:
1000/125

In [ ]:
80*60

In [ ]:
45*8